# Which Numbers are Divisible by 17?

I recently came across 
[this incredible post](https://www.tumblr.com/blog/view/swinginsportscutebasketball/639807401571368960):

> "I'm sorry to let you know that 100,000,001 (one hundred million and one)
> is divisible by 17 and because of that, so is every 16-digit number that 
> is four digits repeated four times e.g. 1234123412341234"
> — [swinginsportscutebasketball](https://www.tumblr.com/blog/view/swinginsportscutebasketball)

That means any 16 digit number of the form a,bcd,abc,dab,cda,bcd where a, b, c, and d are digits is
divisible by 17. This is unintuitive to me, so I wanted to check.

First of all let's use brute force:

In [1]:
from typing import List
from functools import reduce

def check_brute_force() -> None:
    def prepend_digits(strs: List[str], digits: List[str]) -> List[str]:
        joined = map(lambda x: list(map(lambda y: y+x, digits)), strs)
        return list(reduce(lambda a,b: a+b, joined))
    digits = list(map(str, range(10)))
    strs = prepend_digits(digits, digits)
    strs = prepend_digits(strs, digits) 
    strs = prepend_digits(strs, digits)
    strs = list(map(lambda x: x+x+x+x, strs))
    numbers_to_check = list(filter(lambda x: x >= 1000_0000_0000_0000, map(int, strs)))
    failed = False
    error_template = "The number {number} is not divisable by 17. Instead the remainder {number} % 17 is {remainder}"
    for number in numbers_to_check:
        r = number % 17
        if r != 0:
            print(error_template.format(number=number, remainder=remainder))
            failed = True
    if not failed:
        print("All numbers of the form abcd,abcd,abcd,abcd >= 1,000,000,000,000,000 are divisable by 17.")

check_brute_force()

All numbers of the form abcd,abcd,abcd,abcd >= 1,000,000,000,000,000 are divisable by 17.


Okay, so the poster was correct. Technically this brute force check constitutes a proof,
but I'd rather follow the poster's original logic. All four digit numbers of
the form abcd,abcd,abcd,abcd can be written as 

$$ 1,000,100,010,001,000 a + 100,010,001,000,100 b + 10,001,000,100,010 c + 1,000,100,010,001 d $$

where $a,b,c,d \in \mathbb{Z}$ and $1\leq a\leq 9$ and $0\leq b,c,d\leq 9$. If we then define the following
constants, one for each coefficient

$$ A \equiv 1,000,100,010,001,000 $$
$$ B \equiv 100,010,001,000,100 $$
$$ C \equiv 10,001,000,100,010 $$
$$ D \equiv 1,000,100,010,001 $$

and

$$ F \equiv 100,000,001 $$

It is then sufficient to show that $A$,$B$,$C$, and $D$ are divisible by $F$ and that $F$ is divisible by 17. 

In [2]:
from typing import Tuple

def four_digit_coefficients() -> Tuple[int, int, int, int]:
    A = 1_000_100_010_001_000
    B = 100_010_001_000_100
    C = 10_001_000_100_010
    D = 1_000_100_010_001
    return A, B, C, D

def check_divisible_1() -> None:
    A, B, C, D = four_digit_coefficients()
    F = 100_000_001
    ok =  A % F == 0
    ok = ok and B % F == 0
    ok = ok and C % F == 0
    ok = ok and D % F == 0
    ok = ok and F % 17 == 0
    if not ok:
        print("At least one of the remainders was non-zero.")
        return
    print("All passed!")

check_divisible_1()

All passed!


Cool. But this can be simplified further. Note that $C = 10 D$, $B = 10 C$ 
and $A = 10 B$. Therefore, $D$ is the greatest common denominator
of $A$, $B$, $C$, and $D$, all so all it takes to show that 17 divides  
numbers of the form a,bcd,abc,dab,cda,bcd is that it divides $D$.

In [3]:
def check_divisible_2() -> None:
    A, B, C, D = four_digit_coefficients()
    ok = A % B == 0
    ok = ok and B % C == 0
    ok = ok and C % D == 0
    ok = ok and D % 17 == 0
    if not ok:
        print("At least one of the remainders was non-zero.")
        return
    print("All passed!")

check_divisible_2()

All passed!


This method also shows that any number that divides $D$ will also evenly
divide all numbers of the form a,bcd,abc,dab,cda,bcd. 

We can easily construct this list.

In [4]:
from typing import Set

def get_factors(x: int) -> Set[int]:
    ret = set()
    limit = int(x**0.5 + 1)
    for f in range(1, limit):
        if x % f == 0:
            ret.add(f)
            ret.add(int(x / f))
    return ret

def find_other_weird_numbers() -> Set[int]:
    _A, _B, _C, D = four_digit_coefficients()
    return sorted(list(get_factors(D)))

print(find_other_weird_numbers())
            

[1, 17, 73, 137, 1241, 2329, 10001, 170017, 5882353, 100000001, 429411769, 805882361, 7300000073, 13700000137, 58829412353, 1000100010001]


This is a weird list. $D / 73$ is 13700000137 and $D / 137$ is 7300000073.

While we are here though, let's find the numbers that divide numbers of 
the form a,bcd,eab,cde,abc,dea,bcd,eab,cde; 
abc,def,abc,def,abc,def,abc,def,abc,def,abc,def; and so on.

Let the set of numbers of the form
a,bcd,abc,dab,cda,bcd be $\mathcal{S}_4$;
the set of numbers of the form
a,bcd,eab,cde,abc,dea,bcd,eab,cde be $\mathcal{S}_5$; and so on.
Let the set of numbers that evenly divide all elements of
$\mathcal{S}_4$ be $\mathcal{F}_4$;
the set of numbers that evenly divide all elements of
$\mathcal{S}_5$ be $\mathcal{F}_5$; and so on.

Without loss of generality, fix $N \geq 4, N \in \mathbb{Z}$.
Then all of the numbers in $\mathcal{F}_N$ will be divisible by a number

$$ D_N = \sum_{k=0}^{N-1} 10^{Nk} $$

Again, $D_N$ will be the greatest common factor
of elements in $\mathcal{S}_N$.

The set $\mathcal{F}_N$ is then the set of numbers that divide $D_N$.
We can find a lot more interesting numbers automatically.

In [5]:
def print_more_weird_numbers(N: int, F_N: List[int]) -> None:
    margin = 70
    lines = []
    line = ""
    for f in F_N:
        if line == "":
            line = str(f)
            continue
        new_line = line + ", " + str(f)
        if len(new_line) > margin:
            lines.append(line)
            line = str(f)
            continue
        line = new_line
    lines.append(line)
    print("{}:\t{}".format(N, "\n\t".join(lines)))
    
from signal import signal, alarm, SIGALRM # POSIX only

class TimeoutError(Exception):
    pass

def find_even_more_weird_numbers_slow(limit: int) -> None:
    def handler(signum, frame):
        raise TimeoutError("This took to long.")
    signal(SIGALRM, handler)
    alarm(10)
    try:
        for N in range(4, limit):
            D_N = sum([10**(N*k) for k in range(N)])
            F_N = sorted(list(get_factors(D_N)))
            print_more_weird_numbers(N, F_N)
    except TimeoutError as e:
        print(e)
    
find_even_more_weird_numbers_slow(10)

4:	1, 17, 73, 137, 1241, 2329, 10001, 170017, 5882353, 100000001
	429411769, 805882361, 7300000073, 13700000137, 58829412353
	1000100010001
This took to long.


So this computation took too long. We are going to need a more clever factorization system.        

In [12]:
from typing import Dict

class Factorizor:
    def __init__(self) -> None:
        # The known primes. If a number is in this set, it is prime.
        # If it is not in this list, it may still be prime.
        self.prime_cache = {2, 3} 
        # The highest number for which all primes less than the value
        # are already cached. If a number is less than this value
        # and it is not in the prime cache, the number is composite or 1.
        self.prime_cache_limit = 3  
        self.prime_factor_cache = {} # map of ints to their prime factorization
        self.cache_primes_through(1_000_000)
    
    def cache_primes_through(self, x: int) -> None:
        if self.prime_cache_limit >= x:
            return
        candidates = [True for k in range(x)]
        for p in range(2, self.prime_cache_limit):
            if p not in self.prime_cache:
                candidates = False
                continue
            m = p * int(x / p)
            if m == p:
                m += p
            while m < x:
                candiates[m] = False
                m += p
        limit = int(x**0.5 + 1)
        for p in range(self.prime_cache_limit, limit):
            if not candidates[p]:
                continue
            m = 2*p
            while m < x:
                candidates[m] = False
                m += p
        new_primes = filter(lambda p: p >= self.prime_cache_limit and candidates[p], range(x))
        self.prime_cache = self.prime_cache.union(set(new_primes))
        self.prime_cache_limit = x
                
    def known_prime(self, p: int) -> bool:
        return p in self.prime_cache 
        
    def prime_factorization(self, n: int) -> Dict[int, int]:
        if n <= 1:
            raise ValueError("Don't be cute.")
        if n in self.prime_factor_cache:
            return self.prime_factor_cache[n]
        if self.known_prime(n):
            prime_factors = {n: 1}
            self.prime_factor_cache[n] = prime_factors
            return prime_factors
        n_is_prime = True
        limit = int(n**0.5 + 1)
        f1 = 2
        while f1 <= limit:
            if n % f1 != 0:
                f1 += 1
                continue
            if f1 != n:
                n_is_prime = False
            break
        if n_is_prime:
            self.prime_cache.add(n)
            prime_factors = {n: 1}
            self.prime_factor_cache[n] = prime_factors
            return prime_factors
        f2 = int(n / f1)
        prime_factors = {}
        pf1 = self.prime_factorization(f1)
        for prime, count in pf1.items():
            prime_factors[prime] = prime_factors[prime] + count \
                if prime in prime_factors else count
        pf2 = self.prime_factorization(f2)
        for prime, count in pf2.items():
            prime_factors[prime] = prime_factors[prime] + count \
                if prime in prime_factors else count
        
        self.prime_factor_cache[n] = prime_factors 
        return prime_factors
            
                
factorizor = Factorizor()
                
def test_prime_factorization(f: Factorizor) -> None:
    fs = f.prime_factorization(2)
    assert len(fs) == 1
    assert fs[2] == 1
    fs = f.prime_factorization(3)
    assert len(fs) == 1
    assert fs[3] == 1
    fs = f.prime_factorization(15)
    assert len(fs) == 2
    assert fs[3] == 1
    assert fs[5] == 1
    fs = f.prime_factorization(17)
    assert len(fs) == 1
    assert fs[17] == 1
    fs = f.prime_factorization(24)
    assert len(fs) == 2
    assert fs[2] == 3
    assert fs[3] == 1
    print("All passed!")

test_prime_factorization(factorizor)

All passed!


Armed with a slightly faster factor finder, we can look for more weird numbers.

In [13]:
def find_even_more_weird_numbers_fast(limit: int) -> None:
    all_prime_factors = {}
    print("N\tD_N\tPrime Factors")
    for N in range(4, limit):
        D_N = sum([10**(N*k) for k in range(N)])
        all_prime_factors[N] = factorizor.prime_factorization(D_N)
        print("{}\t{}\t{}".format(N, D_N, all_prime_factors[N]))
    print("All Factors")
    for N, prime_factors in all_prime_factors.items():
        primes = sorted(prime_factors.keys())
        counts = [0 for k in range(len(primes))]
        factors = set()
        done = False
        while not done:
            product = 1
            zipped = list(zip(primes, counts))
            for prime, count in zipped:
                product *= prime ** count
            factors.add(product)
            for idx in range(len(zipped)):
                prime, count = zipped[idx]
                if count < prime_factors[prime]:
                    counts[idx] += 1
                    break
                if idx == len(zipped) - 1:
                    done = True
                    break
                counts[idx] = 0
        F_N = sorted(list(factors))
        print_more_weird_numbers(N, F_N)
        
    
find_even_more_weird_numbers_fast(10)

N	D_N	Prime Factors
4	1000100010001	{17: 1, 73: 1, 137: 1, 5882353: 1}
5	100001000010000100001	{21401: 1, 25601: 1, 182521213001: 1}
6	1000001000001000001000001000001	{3: 1, 2: 47, 5: 1, 23: 1, 2377: 1, 8664477161: 1}
7	1000000100000010000001000000100000010000001	{505885997: 1, 2: 58, 3: 3, 59: 1, 4305185420597: 1}
8	100000001000000010000000100000001000000010000000100000001	{17: 1, 2: 130, 5: 1, 101: 1, 2027: 1, 26699: 1, 158129: 1}
9	1000000001000000001000000001000000001000000001000000001000000001000000001	{3: 3, 2: 186, 7: 3, 37409: 1, 29429767: 1}
All Factors
4:	1, 17, 73, 137, 1241, 2329, 10001, 170017, 5882353, 100000001
	429411769, 805882361, 7300000073, 13700000137, 58829412353
	1000100010001
5:	1, 21401, 25601, 547887001, 182521213001, 3906136479434401
	4672725574038601, 100001000010000100001
6:	1, 2, 3, 4, 5, 6, 8, 10, 12, 15, 16, 20, 23, 24, 30, 32, 40, 46, 48
	60, 64, 69, 80, 92, 96, 115, 120, 128, 138, 160, 184, 192, 230, 240
	256, 276, 320, 345, 368, 384, 460, 480, 512, 55

Well, I was hoping that these numbers would be more interesting.
I am not seeing any patterns worth investigating further 🤷 